In [20]:
import pandas as pd

In [14]:
#This is a dataset of fake student survey responses generated with ChatGPT
surveys = pd.read_excel("fake_student_survey_data.xlsx")
text = surveys['text']
surveys.head()

,text,Academic,Facilities,Career,Financial,Diversity,Wellness,Social,Technology,Sustainability,Policy,Communication,Ethics
0,"""Create innovation hubs that support entre...",0,0,1,0,0,0,0,0,0,0,0,0
1,"""Create safe social spaces for students to...",0,0,0,0,0,0,1,0,0,0,0,0
2,"""Develop leadership programs focused on pu...",0,0,1,0,0,0,0,0,0,0,0,0
3,"""Develop mentorship programs that connect ...",0,0,1,0,0,0,0,0,0,0,0,0
4,"""Encourage interdisciplinary collaboration...",0,0,0,0,0,0,1,0,0,0,0,0


In [68]:
#Converting dataframe to Huggingface Dataset
#We're going to focus on the career label for binary classification
career_df = surveys[['text', 'Career']]
career_df.rename(columns={'Career': 'label'}, inplace=True)

from datasets import Dataset
train_career = Dataset.from_pandas(career_df)
train_career[0]

C:\Users\kevin\AppData\Local\Temp\ipykernel_39580\3143108953.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_df.rename(columns={'Career': 'label'}, inplace=True)


{'text': '    "Create innovation hubs that support entrepreneurial ventures. Pursuing my business ideas was challenging without dedicated resources and mentorship."',
 'label': 1}

In [28]:
#Here is a eval dataset of fake student survey responses generated with Chat GPT to test a few binary classification models
surveys_eval = pd.read_excel("fake_survey_eval_data.xlsx")
eval_text = surveys_eval['text']
surveys_eval.head()

,text,Career,Social,Diversity
0,Offer workshops that teach specific skills rel...,1,0,0
1,Increase partnerships with companies to provid...,1,0,0
2,Provide personalized career counseling session...,1,0,0
3,Integrate courses that prepare students for in...,1,0,0
4,Create mentorship programs where students can ...,1,0,0


In [69]:
career_eval = surveys_eval[['text', 'Career']]
career_eval.rename(columns={'Career': 'label'}, inplace=True)

eval_career = Dataset.from_pandas(career_eval)
eval_career[0]

C:\Users\kevin\AppData\Local\Temp\ipykernel_39580\595259346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_eval.rename(columns={'Career': 'label'}, inplace=True)


{'text': 'Offer workshops that teach specific skills relevant to different industries, such as project management, data analysis, or digital marketing.',
 'label': 1}

In [175]:
#choose model
#model_path = "roberta-base"
model_path = "microsoft/deberta-v3-base"

In [165]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

C:\Users\kevin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kevin\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

C:\Users\kevin\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [166]:
def preprocess_function(surveys):
    return tokenizer(surveys['text'], truncation=True)

In [167]:
tokenized_surveys = train_career.map(preprocess_function)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [169]:
#Here is what our data looks like after tokenization
#tokenized_surveys[0]

In [170]:
tokenized_eval = eval_career.map(preprocess_function)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [171]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [172]:
import evaluate

accuracy = evaluate.load("accuracy")

In [173]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [174]:
id2label = {0: "Not Career Related", 1: "Career Related"}
label2id = {"Not Career Related": 0, "Career Related": 1}

In [179]:
#Import the necessary packages 
#We're going to test both RoBERTa and DeBERTa for binary classification
#You can change the model by updating the model_path variable above

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_path
    ,num_labels=2 
    ,id2label=id2label
    ,label2id=label2id
)

pytorch_model.bin:  92%|#########1| 341M/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [180]:
training_args = TrainingArguments(
    output_dir="deberta_survey_example",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=8,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_surveys,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.649381,0.666667
2,No log,0.733775,0.666667
3,No log,0.629783,0.666667
4,No log,0.603087,0.666667
5,No log,0.581126,0.700000
6,No log,0.677386,0.750000
7,No log,0.758316,0.750000
8,No log,0.801666,0.750000


TrainOutput(global_step=184, training_loss=0.27229918604311737, metrics={'train_runtime': 1290.8982, 'train_samples_per_second': 0.558, 'train_steps_per_second': 0.143, 'total_flos': 11777514272328.0, 'train_loss': 0.27229918604311737, 'epoch': 8.0})

In [181]:
#Test out some predictions -- on a very obvious career example
from transformers import pipeline
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
answer = clf("We should focus on learning skills for our career.")
answer2 = clf("There should be more snacks available between classes")
print(answer)
print(answer2)

[{'label': 'Not Career Related', 'score': 0.6981530785560608}]
[{'label': 'Not Career Related', 'score': 0.923100471496582}]


In [182]:
#Since this dataset is unbalanced (only mentions career stuff about 10% of the time), let's add class weights 
#to assign higher importance to the minority class

#Resetting the model
weighted_model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=2, id2label=id2label, label2id=label2id
)

from torch import nn
import torch

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # Here we update the loss function 
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9])) #Set class weights here
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

weighted_trainer = CustomTrainer(
    model=weighted_model,
    args=training_args,
    train_dataset=tokenized_surveys,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [183]:
weighted_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.726213,0.333333
2,No log,0.696942,0.366667
3,No log,0.646362,0.666667
4,No log,0.846927,0.733333
5,No log,0.745866,0.816667
6,No log,0.844164,0.800000
7,No log,0.834919,0.816667
8,No log,0.836908,0.816667


TrainOutput(global_step=184, training_loss=0.31678048424098804, metrics={'train_runtime': 902.719, 'train_samples_per_second': 0.798, 'train_steps_per_second': 0.204, 'total_flos': 11777514272328.0, 'train_loss': 0.31678048424098804, 'epoch': 8.0})

In [184]:
#Test out some predictions -- on a very obvious career example
from transformers import pipeline
weighted_clf = pipeline("text-classification", model=weighted_model, tokenizer=tokenizer)
weighted_answer = weighted_clf("We should focus on learning skills for our career.")
weighted_answer2 = weighted_clf("There should be more snacks available between classes")
print(weighted_answer)
print(weighted_answer2)

[{'label': 'Career Related', 'score': 0.6545425653457642}]
[{'label': 'Not Career Related', 'score': 0.5999439358711243}]
